In [1]:
from experiment import *
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [2]:
filename = "./Data/settles.acl16.learning_traces.13m.csv.gz"
method = "custom"


trainset, testset = read_data(filename, method)

reading data...0...

Seconds: 27649635
Days: 320.0189236111111
Right: 4
Wrong: 2
Session Right: 2
Session Wrong: 0
Seconds: 160
Days: 0.001851851851851852
Right: 7
Wrong: 0
Session Right: 0
Session Wrong: 1


1000000...

Seconds: 422909
Days: 4.894780092592592
Right: 25
Wrong: 6
Session Right: 1
Session Wrong: 0


2000000...

...

10000000...

Seconds: 102
Days: 0.0011805555555555556
Right: 6
Wrong: 3
Session Right: 3
Session Wrong: 0


11000000...

Seconds: 100252
Days: 1.1603240740740741
Right: 5
Wrong: 0
Session Right: 1
Session Wrong: 0


12000000...done!


In [6]:
print(trainset[0].fv)

[('right', 2.0), ('wrong', 1.0), ('session_right', 1.4142135623730951), ('session_wrong', 1.0), ('bias', 1.0), ('es:bajo/bajo<pr>', 1.0)]


In [30]:
featuretags = ["session_right", "session_wrong", "right", "wrong"]

class TrainDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, ind):
        vector = []
        for i in self.data[ind].fv:
            if i[0] in featuretags:
                vector.append(i[1])
        x = np.array(vector)
        y = np.array(self.data[ind].h)
        return x, y
class TestDataset(TrainDataset):
    def __getitem__(self, ind):
        vector = []
        for i in self.data[ind].fv:
            if i[0] in featuretags:
                vector.append(i[1])
        x = np.array(vector)
        y = np.array(self.data[ind].h)
        return x, y

validx = int(len(trainset)*.8)
train_set = TrainDataset(trainset[:validx])

val_set = TrainDataset(trainset[validx:])
test_set  = TestDataset(testset)

batch_size = 512
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)


In [31]:
x, y = train_set.__getitem__(0)
print(x)

for batch_num, input_data in enumerate(train_loader):
        
        #zero gradient
        optimizer.zero_grad()
        x, y = input_data
        print(len(x))
        print(x.shape)
        break

[2.         1.         1.41421356 1.        ]
512
torch.Size([512, 4])


In [32]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_dim = 4
        self.hidden_dim = 32
        self.output_dim = 1
        
        
        self.input = nn.Linear(self.input_dim,self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)
    
    def forward(self, x):
        hidden = self.input(x)
        out = self.output(hidden)
        return out

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

In [34]:
epochs = 50

model.train()

# iterate through epochs
for epoch in range(epochs):
    losses = []
    vallosses = []
    # iterate through batches
    for batch_num, input_data in enumerate(train_loader):
        
        #zero gradient
        optimizer.zero_grad()
        x, y = input_data
        
        # send batch to GPU
        x = x.to(device).float()
        y = y.to(device).float()
        
        # generate predictions
        output = model(x)
        
        # compute loss
        loss = criterion(output, y)
        
        # compute gradients
        loss.backward()
#         losses.append(loss.item())
        
        # step parameters
        optimizer.step()
    if epoch % 5 == 0:
        losses.append(loss.item())
        for batch, data in enumerate(val_loader):
            x,y = data
            model.eval()
            output = model(x)
            loss = criterion(output,y)
            vallosses.append(loss.item())
    print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))

plt.plot(np.arange(len(losses)), losses)
plt.plot(np.arange(len(losses)), vallosses)

C:\Users\alexo\Anaconda3\envs\cs474\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([425])) that is different to the input size (torch.Size([425, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Tensor for argument #2 'mat1' is on CPU, but expected it to be on GPU (while checking arguments for addmm)